# RomKorr – Map Builder (v2)

Änderungen gegenüber v1:
- **Hotspots immer klickbar**: eigene Leaflet-Panes mit Z-Index (Hotspots über Heatmap).
- **Popup-Texte**: „Korrespondenzen:“ + Beispiele im Format „Sender → Empfänger, DD.MM.YYYY, Original“.
- **Personen-Filter**: Sidebar-Dropdown (Top-N Personen nach Häufigkeit) filtert **Linien** clientseitig (ohne Backend).

Output:
- `outputs/romkorr_map.html`

In [1]:
from __future__ import annotations

from pathlib import Path
import json
import numpy as np
import pandas as pd

import folium
from folium.plugins import HeatMap, MarkerCluster
from branca.element import MacroElement, Template

In [2]:
PROJECT_ROOT = Path.cwd()

MASTER = PROJECT_ROOT / "data" / "processed" / "letters_master.parquet"
PLACES = PROJECT_ROOT / "data" / "processed" / "places_agg.parquet"

OUT_DIR = PROJECT_ROOT / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_HTML = OUT_DIR / "romkorr_map.html"

# Karte
CENTER = (51.0, 10.0)
ZOOM_START = 6

# Linien
SAMPLE_FRAC = 1.0   # 1.0 = alle Linien; setze z.B. 0.3 für schnelleres Bauen

# Personenfilter
TOP_N_PERSONS = 60  # Dropdown enthält die häufigsten Personen (Sender+Recipient)

In [3]:
letters = pd.read_parquet(MASTER)
places = pd.read_parquet(PLACES)

letters.head()

,Date,Sender,Recipient,place_dispatch,place_destination,dispatch_geonames_url,destination_geonames_url,link,dispatch_lat,dispatch_lon,destination_lat,destination_lon,distance_km,letter_id,link_canonical,date_parsed,date_iso
0,"Donnerstag, 6. Januar 1791",August Wilhelm von Schlegel,Christian Gottlob Heyne,Göttingen,Göttingen,https://www.geonames.org/2918632,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/1?...,51.53443,9.93228,51.53443,9.93228,0.000000,1,https://briefe-der-romantik.de/letters/view/1?...,1791-01-06,1791-01-06
1,"Mittwoch, 20. September 1797",August Wilhelm von Schlegel,Christian Gottlob Heyne,Jena,Göttingen,https://www.geonames.org/2895044,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/2?...,50.92878,11.58990,51.53443,9.93228,133.622388,2,https://briefe-der-romantik.de/letters/view/2?...,1797-09-20,1797-09-20
2,"Samstag, 26. Mai 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Berlin,Leipzig,https://www.geonames.org/2950159,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/3?...,52.52437,13.41053,51.33962,12.37129,149.769217,3,https://briefe-der-romantik.de/letters/view/3?...,1798-05-26,1798-05-26
3,"Mittwoch, 31. Oktober 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Jena,Leipzig,https://www.geonames.org/2895044,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/4?...,50.92878,11.58990,51.33962,12.37129,71.129758,4,https://briefe-der-romantik.de/letters/view/4?...,1798-10-31,1798-10-31
4,[Mitte August 1801],Sophie Bernhardi,August Wilhelm von Schlegel,Berlin,Jena,https://www.geonames.org/2950159,https://www.geonames.org/2895044,https://briefe-der-romantik.de/letters/view/5?...,52.52437,13.41053,50.92878,11.58990,217.247072,5,https://briefe-der-romantik.de/letters/view/5?...,1801-08-01,1801-08-01


In [4]:
def to_de_date(date_iso: str) -> str:
    if not isinstance(date_iso, str) or not date_iso:
        return "?"
    try:
        y, m, d = date_iso.split("-")
        return f"{d}.{m}.{y}"
    except Exception:
        return "?"

# Häufigkeiten für Personenfilter (Sender + Recipient)
person_counts = (pd.concat([letters["Sender"], letters["Recipient"]], ignore_index=True)
                 .fillna("Unknown"))
person_counts = person_counts[person_counts.ne("Unknown")]
top_persons = person_counts.value_counts().head(TOP_N_PERSONS)
top_persons.head(10), top_persons.shape

(August Wilhelm von Schlegel    1342
 Friedrich Schleiermacher       1138
 Friedrich von Schlegel          966
 Johann Gottlieb Fichte          944
 Caroline von Schelling          399
 Dorothea von Schlegel           232
 Johanna Fichte                  202
 Samuel Ernst Stubenrauch        202
 Lotte Schleiermacher            106
 Johann Wolfgang von Goethe      100
 Name: count, dtype: int64,
 (60,))

## 1) Map + Panes (Layer-Reihenfolge)

Heatmap/Lines/Hotspots erhalten eigene Panes mit definiertem Z-Index. Dadurch liegen Hotspots **immer über** der Heatmap und bleiben klickbar.

In [5]:
m = folium.Map(location=CENTER, zoom_start=ZOOM_START, control_scale=True, prefer_canvas=True)

folium.map.CustomPane("heatmap", z_index=200).add_to(m)
folium.map.CustomPane("lines", z_index=300).add_to(m)
folium.map.CustomPane("hotspots", z_index=500).add_to(m)

m

## 2) Heatmap

Leaflet-Heatmap wirkt bei starkem Zoom naturgemäß „punktiger“. Wenn dich das stört, können wir als nächsten Schritt zwei Heatmap-Layer (weit/nah) anbieten.

In [6]:
heat_points = places.groupby(["lat","lon"], as_index=False)["count"].sum()

heat_layer = folium.FeatureGroup(name="Heatmap", show=True)
HeatMap(
    data=heat_points[["lat","lon","count"]].values.tolist(),
    radius=18,
    blur=20,
    max_zoom=7
).add_to(heat_layer)
heat_layer.add_to(m)

heat_points.sort_values("count", ascending=False).head(10)

,lat,lon,count
130,52.52437,13.41053,2366
61,50.92878,11.58990,2070
74,51.05089,13.73832,326
126,52.37403,4.88969,298
97,51.33962,12.37129,293
133,52.73679,15.22878,259
10,47.36667,8.55000,251
65,50.98030,11.32903,244
123,52.26594,10.52673,233
125,52.37052,9.73322,211


## 3) Hotspots (aggregiert)

Ein Marker pro Koordinate. Popup zeigt Count + Beispiele. Hotspots liegen in eigener Pane über der Heatmap.

In [7]:
hotspots_layer = folium.FeatureGroup(name="Hotspots", show=True)
cluster = MarkerCluster(name="Hotspot-Cluster").add_to(hotspots_layer)

examples_by_k = {}
def k_from(lat, lon):
    return (float(lat), float(lon))

def add_examples(df: pd.DataFrame, lat_col: str, lon_col: str):
    sub = df.dropna(subset=[lat_col, lon_col]).copy()
    sub["k"] = list(zip(sub[lat_col].astype(float), sub[lon_col].astype(float)))
    sub = sub.sort_values(["date_iso"], na_position="last")
    for k0, grp in sub.groupby("k"):
        if k0 not in examples_by_k:
            examples_by_k[k0] = []
        for _, r in grp.head(6).iterrows():
            if len(examples_by_k[k0]) >= 6:
                break
            examples_by_k[k0].append(r)

add_examples(letters, "dispatch_lat", "dispatch_lon")
add_examples(letters, "destination_lat", "destination_lon")

place_name_by_k = {}
tmp = places.copy()
tmp["k"] = list(zip(tmp["lat"].astype(float), tmp["lon"].astype(float)))
for k0, grp in tmp.groupby("k"):
    s = grp["place_mode"].dropna()
    place_name_by_k[k0] = (s.mode().iloc[0] if len(s.mode()) else "Ort (unbekannt)")

for _, r in heat_points.sort_values("count", ascending=False).iterrows():
    k0 = k_from(r["lat"], r["lon"])
    cnt = int(r["count"])
    place_name = place_name_by_k.get(k0, "Ort (unbekannt)")

    ex = examples_by_k.get(k0, [])
    ex_items = []
    for e in ex:
        date_de = to_de_date(e.get("date_iso", ""))
        ex_items.append(
            f"<li>{e.get('Sender','?')} → {e.get('Recipient','?')}, {date_de}, "
            f"<a href='{e.get('link_canonical','#')}' target='_blank'>Original</a></li>"
        )

    popup_html = (
        "<div style='max-width: 380px;'>"
        f"<h4 style='margin:0 0 6px 0;'>{place_name}</h4>"
        f"<div><b>Korrespondenzen:</b> {cnt}</div>"
        "<hr style='margin:8px 0;' />"
        "<div><b>Beispiele:</b></div>"
        "<ul style='padding-left: 18px; margin: 6px 0;'>"
        + ("".join(ex_items) if ex_items else "<li>Keine Beispiele verfügbar.</li>")
        + "</ul>"
        "</div>"
    )

    folium.CircleMarker(
        location=[float(r["lat"]), float(r["lon"])],
        radius=max(3, min(18, np.log1p(cnt) * 3.0)),
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=420),
        pane="hotspots",
    ).add_to(cluster)

hotspots_layer.add_to(m)

## 4) Linien + Personenfilter

Wir zeichnen Linien in JavaScript (Leaflet) und filtern sie clientseitig per Dropdown. Dadurch hängen wir nicht mehr an tausenden Python/Folium-Objekten.

- Default: alle Linien sichtbar
- Dropdown: matcht Sender **oder** Recipient

Das ist für eine statische HTML die beste Balance aus Funktion & Performance.

In [8]:
lines_df = letters.dropna(subset=["dispatch_lat","dispatch_lon","destination_lat","destination_lon"]).copy()

if 0 < SAMPLE_FRAC < 1:
    rng = np.random.default_rng(42)
    lines_df = lines_df.loc[rng.random(len(lines_df)) < SAMPLE_FRAC].copy()

lines_df["date_de"] = lines_df["date_iso"].map(to_de_date)

lines_payload = []
for _, r in lines_df.iterrows():
    lines_payload.append({
        "a": [float(r["dispatch_lat"]), float(r["dispatch_lon"])],
        "b": [float(r["destination_lat"]), float(r["destination_lon"])],
        "s": str(r.get("Sender","?")),
        "r": str(r.get("Recipient","?")),
        "d": str(r.get("date_de","?")),
        "km": float(r.get("distance_km", float("nan"))) if pd.notnull(r.get("distance_km")) else None,
        "u": str(r.get("link_canonical","#")),
        "id": int(r.get("letter_id")) if pd.notnull(r.get("letter_id")) else None,
    })

persons_list = [{"name": k, "count": int(v)} for k, v in top_persons.items()]

len(lines_payload), persons_list[:5]

(4385,
 [{'name': 'August Wilhelm von Schlegel', 'count': 1342},
  {'name': 'Friedrich Schleiermacher', 'count': 1138},
  {'name': 'Friedrich von Schlegel', 'count': 966},
  {'name': 'Johann Gottlieb Fichte', 'count': 944},
  {'name': 'Caroline von Schelling', 'count': 399}])

In [9]:
template = Template(r"""
{% macro html(this, kwargs) %}

<style>
  .rk-sidebar {
    position: absolute;
    top: 10px;
    left: 10px;
    z-index: 9999;
    background: rgba(255,255,255,0.92);
    padding: 10px 12px;
    border-radius: 10px;
    box-shadow: 0 2px 10px rgba(0,0,0,.15);
    width: 320px;
    font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;
  }
  .rk-sidebar h3 { margin: 0 0 8px 0; font-size: 14px; }
  .rk-row { display:flex; gap:8px; align-items:center; }
  .rk-row select { width: 100%; padding: 6px; border-radius: 8px; border: 1px solid #ddd; }
  .rk-row button { padding: 6px 10px; border-radius: 8px; border: 1px solid #ddd; background: #fff; cursor:pointer; }
  .rk-meta { font-size: 12px; opacity: .8; margin-top: 6px; }
</style>

<div class="rk-sidebar">
  <h3>Filter: Korrespondenzen nach Person</h3>
  <div class="rk-row">
    <select id="rkPerson">
      <option value="">(alle)</option>
    </select>
    <button id="rkReset" title="Filter zurücksetzen">Reset</button>
  </div>
  <div class="rk-meta">
    Linien: <span id="rkLineCount">0</span> (von <span id="rkLineTotal">0</span>)<br/>
    Hinweis: Filter matcht Sender <i>oder</i> Recipient.
  </div>
</div>

<script>
(function() {
  const map = {{this._parent.get_name()}};

  const LINES = {{ this.lines_json }};
  const PERSONS = {{ this.persons_json }};

  const linesLayer = L.layerGroup([], {pane: 'lines'}).addTo(map);

  function popupHtml(x) {
    const km = (x.km === null || x.km === undefined) ? "?" : (Math.round(x.km * 10) / 10).toFixed(1);
    return `
      <div style="max-width: 360px;">
        <b>${x.s}</b> → <b>${x.r}</b><br/>
        ${x.d} &nbsp;·&nbsp; Distanz: ${km} km<br/>
        <a href="${x.u}" target="_blank">Original</a>
      </div>`;
  }

  function drawLines(filterName) {
    linesLayer.clearLayers();
    let shown = 0;

    for (let i=0; i<LINES.length; i++) {
      const x = LINES[i];
      if (filterName) {
        if (x.s !== filterName && x.r !== filterName) continue;
      }
      const line = L.polyline([x.a, x.b], {
        weight: 1,
        opacity: 0.35,
        pane: 'lines'
      });
      line.bindPopup(popupHtml(x), {maxWidth: 380});
      line.addTo(linesLayer);
      shown++;
    }
    document.getElementById('rkLineCount').textContent = shown.toString();
  }

  const sel = document.getElementById('rkPerson');
  PERSONS.forEach(p => {
    const opt = document.createElement('option');
    opt.value = p.name;
    opt.textContent = `${p.name} (${p.count})`;
    sel.appendChild(opt);
  });

  document.getElementById('rkLineTotal').textContent = LINES.length.toString();

  sel.addEventListener('change', () => drawLines(sel.value));
  document.getElementById('rkReset').addEventListener('click', () => {
    sel.value = "";
    drawLines("");
  });

  drawLines("");
})();
</script>

{% endmacro %}
""")

macro = MacroElement()
macro._template = template
macro.lines_json = json.dumps(lines_payload, ensure_ascii=False)
macro.persons_json = json.dumps(persons_list, ensure_ascii=False)

m.get_root().add_child(macro)

In [10]:
folium.LayerControl().add_to(m)
m.save(str(OUT_HTML))
OUT_HTML

WindowsPath('g:/Meine Ablage/CodingProjekte/RomKorr/RomKorr/outputs/romkorr_map.html')